### load your libraries

In [3]:
import pandas as pd #ingesting and transforming raw data
import ast #abstract syntax tree: converts string to python object

### load the dataset

In [5]:
customers = pd.read_csv("./data/customers.csv")
offers = pd.read_csv("./data/offers.csv")
events = pd.read_csv("./data/events.csv")

### Data exploration
* size of data
* data types
* observe inconsistencies in the dataset

In [8]:
# Size of the dataframe
tables = [customers, offers, events]
for i in tables:
    size = i.shape
    print(size)

(17000, 5)
(10, 6)
(306534, 4)


In [9]:
# Data types
# For customers
customers.dtypes

customer_id             str
became_member_on      int64
gender                  str
age                   int64
income              float64
dtype: object

In [10]:
# Data types
# For events
events.dtypes

customer_id      str
event            str
value            str
time           int64
dtype: object

In [11]:
# Data types
# For offers
offers.dtypes

offer_id        str
offer_type      str
difficulty    int64
reward        int64
duration      int64
channels        str
dtype: object

In [21]:
# Observe inconsistencies'
# for customers
customers.head()

,customer_id,became_member_on,gender,age,income
0,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,118,65405.0
1,0610b486422d4921ae7d2bf64640c50b,2017-07-15,F,55,112000.0
2,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN,118,65405.0
3,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,F,75,100000.0
4,a03223e636434f42ac4c3df47e8bac43,2017-08-04,NaN,118,65405.0


In [13]:
# Observe inconsistencies'
# for offers
events.head()

,customer_id,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [14]:
# Observe inconsistencies'
# for offers
offers.head()

,offer_id,offer_type,difficulty,reward,duration,channels
0,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,10,7,"['email', 'mobile', 'social']"
1,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,10,5,"['web', 'email', 'mobile', 'social']"
2,3f207df678b143eea3cee63160fa8bed,informational,0,0,4,"['web', 'email', 'mobile']"
3,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5,5,7,"['web', 'email', 'mobile']"
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,20,5,10,"['web', 'email']"


## Customer Table Transformation

In [15]:
# Convert the became_member_on column to datetime format
customers['became_member_on'] = pd.to_datetime(customers['became_member_on'], format="%Y%m%d")

In [20]:
# Fill the null value in the income column with the average income
avg_income = float(customers['income'].mean())
avg_income = round(avg_income, 1)

# customers['income'].fillna(avg_income, inplace=True)
customers['income'] = customers['income'].fillna(avg_income)

## Offers Table Transformation

### We want to create a bridge tble that satisfies 2NF

In [23]:
# Converting the strings to an original list

offers['channels'] = offers['channels'].apply(ast.literal_eval)

In [24]:
# explode the dataframe to meet the 1NF requirement
# create a junction table
offers_channels = offers[['offer_id', 'channels']]
offers_channels = offers_channels.explode('channels')

In [25]:
offers_transformed = offers.drop(columns=['channels'])
offers_transformed


,offer_id,offer_type,difficulty,reward,duration
0,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,10,7
1,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,10,5
2,3f207df678b143eea3cee63160fa8bed,informational,0,0,4
3,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5,5,7
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,20,5,10
5,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,7,3,7
6,fafdcd668e3743c1bb461111dcafc2a4,discount,10,2,10
7,5a8bc65990b245e5a138643cd4eb9837,informational,0,0,3
8,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5,5,5
9,2906b810c7d4411798c6938adc9daaa5,discount,10,2,7
